# DS0-560 Final Project Part 2
- Chia-Chi (Rachel) Sung
- Bo Kyung Cho
- Hsiao-Fang (Kathy) Chen
- Alexander Furrow
- Ming-Hsien (Sandra) Chen

## Data Preprocessing

### Import Package

In [1]:
import pandas as pd
import numpy as np
import nltk
import re

### Import Data

In [2]:
# Read outfit_combinations
outfit = pd.read_csv("/Users/rachel/Desktop/560Project-part2/Part 2/outfit_combinations.csv")
outfit.head(3)

,outfit_id,product_id,outfit_item_type,brand,product_full_name
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel


In [3]:
# Read full_data
full = pd.read_csv("/Users/rachel/Desktop/560Project-part2/Part 2/full_data.csv")

# Drop unused columns
full = full.drop(["mpn", "created_at", "updated_at", "deleted_at", "brand_canonical_url", \
                  "labels", "bc_product_id"], axis=1)

In [4]:
# Read extra_data
extra = pd.read_csv("/Users/rachel/Downloads/extra_data.csv")

# Drop unused columns
extra = extra.drop(["mpn", "created_at", "updated_at", "saleable", "brand_canonical_url", \
                    "labels", "bc_product_id", "notes"], axis=1)

# Rename column
extra = extra.rename(columns={"name": "product_full_name"})

# Reorder columns (the same as full_data)
extra = extra[['product_id', 'brand', 'product_full_name', 'description', 'brand_category', 'details']]

extra.head(1)

,product_id,brand,product_full_name,description,brand_category,details
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,A.L.C.,Lennox High Waist Cotton & Linen Pants,High-rise trousers tailored from a cool Italia...,Unknown,"True to size. High rise.\n31"" inseam; 14"" leg ..."


In [5]:
# Concat full_data and extra_data

fullextra = pd.concat([full, extra])
fullextra.head(5)

,product_id,brand,product_full_name,description,brand_category,details
0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,"A modern pump, in a rounded silhouette with an..."
1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,Dress it down with jeans and sneakers or dress...
2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,100% UV protection Case and cleaning cloth inc...
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",Canvas upper Round toe Lace-up vamp SmartFOAM ...
4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,100% UV protection Gradient lenses Adjustable ...


In [6]:
df = pd.merge(fullextra, outfit, on=["product_id"], how='inner')
df.shape

(15363, 10)

In [7]:
# Rename columns
df = df.rename(columns={"brand_x": "brand", "product_full_name_x": "product_full_name"})

# Drop unused columns
df = df.drop(["brand_y", "product_full_name_y"], axis=1)

# Drop duplicates
df = df.drop_duplicates(subset=["product_id", "outfit_id"], keep="first")

df.head(3)

,product_id,brand,product_full_name,description,brand_category,details,outfit_id,outfit_item_type
0,01DVA59VHYAPT4PVX32NXW91G5,Tibi,Juan Embossed Mules,Tibi's Juan embossed mules are made from shiny...,women:SHOES:MULES,As seen on the Pre-Fall ‘19 runway Heel measur...,01DVA879D7TQ59VPTTGCMJWWSK,shoe
3,01DVA4XY7A0QMMSK3V3SBR52J9,Alexandre Birman,Clarita Bow-Embellished Suede Sandals,Alexandre Birman's 'Clarita' sandals have quic...,women:SHOES:SANDALS,Heel height measures approximately 50mm / 2 in...,01DVA8GAYP45BCEMYGEK7FXGDQ,shoe
4,01DVA4XY7A0QMMSK3V3SBR52J9,Alexandre Birman,Clarita Bow-Embellished Suede Sandals,Alexandre Birman's 'Clarita' sandals have quic...,women:SHOES:SANDALS,Heel height measures approximately 50mm / 2 in...,01DWJE4FDNYRV6ZJBG25HJFYY2,shoe


### Data Cleaning

In [8]:
df["product_id"] = df["product_id"].str.upper()
df["brand"] = df["brand"].str.lower()
df['product_full_name'] = df["product_full_name"].str.lower()
df["description"] = df["description"].str.lower()
df["description"] = df["description"].str.replace("\n", " ")
df["brand_category"] = df["brand_category"].str.title()
df["brand_category"] = df["brand_category"].str.replace(r"(/|,|:)", " ")
df["brand_category"] = df["brand_category"].str.lower()
df["details"] = df["details"].str.title()
df["details"] = df["details"].str.replace("\n", " ")
df["outfit_id"] = df["outfit_id"].str.upper()
df["outfit_item_type"] = df["outfit_item_type"].str.lower()
df = df.replace(np.nan, "", regex=True)
df = df.replace(r"(unknown|Unknown)", "", regex=True)
df.reset_index(inplace = True)

df['outfit_id']= df['outfit_id'].astype(str)
df['product_id']= df['product_id'].astype(str)

In [9]:
# Merge columns: brand, product_full_name, description, details, outfit_item_type

df["new_column"] = df["brand"] + " " + df["product_full_name"] + " " + df["description"] + " " + df["details"] + " " + df["outfit_item_type"]
df.head(3)

,index,product_id,brand,product_full_name,description,brand_category,details,outfit_id,outfit_item_type,new_column
0,0,01DVA59VHYAPT4PVX32NXW91G5,tibi,juan embossed mules,tibi's juan embossed mules are made from shiny...,women shoes mules,As Seen On The Pre-Fall ‘19 Runway Heel Measur...,01DVA879D7TQ59VPTTGCMJWWSK,shoe,tibi juan embossed mules tibi's juan embossed ...
1,3,01DVA4XY7A0QMMSK3V3SBR52J9,alexandre birman,clarita bow-embellished suede sandals,alexandre birman's 'clarita' sandals have quic...,women shoes sandals,Heel Height Measures Approximately 50Mm / 2 In...,01DVA8GAYP45BCEMYGEK7FXGDQ,shoe,alexandre birman clarita bow-embellished suede...
2,4,01DVA4XY7A0QMMSK3V3SBR52J9,alexandre birman,clarita bow-embellished suede sandals,alexandre birman's 'clarita' sandals have quic...,women shoes sandals,Heel Height Measures Approximately 50Mm / 2 In...,01DWJE4FDNYRV6ZJBG25HJFYY2,shoe,alexandre birman clarita bow-embellished suede...


In [10]:
# Merge columns:outfit_item_type, product_full_name, product_id

df['output_name']=df['outfit_item_type']+':'+df['product_full_name']+'('+df['product_id']+')'

In [11]:
# Tokenize the sentences into words

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")
df["new_column"] = df["new_column"].apply(lambda x: tokenizer.tokenize(x))

In [12]:
# Remove stopwords

from nltk.corpus import stopwords
stop = stopwords.words('english')

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

cleaned = []
for item in df["new_column"]:
    new_words = []
    for word in item:
        if word in stop:
            continue
        new_words.append(word)

    cleaned.append(new_words)

df["new_column"] = cleaned

In [13]:
# Lemmatize

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    print(text)
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

df["new_column"] = df["new_column"].apply(lambda x: word_lemmatizer(x))

['tibi', 'juan', 'embossed', 'mules', "tibi's", 'juan', 'embossed', 'mules', 'made', 'shiny', 'black', 'leather', 'feature', 'kitten', 'heel', 'minimalist', 'silhouette', 'also', 'available', 'snakeskin', 'burgundy', 'suede', 'As', 'Seen', 'On', 'The', 'Pre-Fall', '19', 'Runway', 'Heel', 'Measures', 'Approximately', '35Mm', '1', '3In', 'Slip', 'On', 'Composition', 'Leather', 'Color', 'Black', 'Made', 'In', 'Italy', 'This', 'Style', 'Fits', 'True', 'To', 'Size', 'We', 'Suggest', 'Ordering', 'Your', 'Normal', 'Size', 'shoe']
['alexandre', 'birman', 'clarita', 'bow-embellished', 'suede', 'sandals', 'alexandre', "birman's", 'clarita', 'sandals', 'quickly', 'risen', 'cult', 'status', 'among', 'fashion', 'glitterati', 'classic', 'double', 'strap', 'construction', 'embellished', "label's", 'signature', 'knot', 'detailing', 'suede', 'rendition', 'perfect', 'outdoor', 'weddings', 'parties', 'Heel', 'Height', 'Measures', 'Approximately', '50Mm', '2', 'Inches', 'Ties', 'At', 'Ankle', 'Composition

['toteme', 'ossi', 'wool-blend', 'high-rise', 'trousers', 'designed', 'curated', 'minimalistic', 'style', 'icon', 'elin', 'kling', 'totême', 'provides', 'updated', 'essentials', 'effortless', 'mix', 'match', 'ideal', 'work', 'weekends', 'high-rise', 'ossi', 'tailored', 'full', 'length', 'tailored', 'leg', 'wear', 'matching', 'blazer', 'smart', 'look', 'keep', 'casual', 'oversized', 'knit', 'sneakers', 'Button', 'And', 'Concealed', 'Zip', 'Fastening', 'At', 'Front', 'Composition', '55', 'Polyamide', '45', 'Wool', 'Imported', 'This', 'Style', 'Fits', 'True', 'To', 'Sizewe', 'Suggest', 'Taking', 'Your', 'Normal', 'Size', 'Model', 'Measurements', 'Height', '5', '9', '175Cm', 'Bust', '32', '81', '2Cm', 'Waist', '24', '60', '96Cm', 'Hip', '34', '86', '36Cmshe', 'Is', 'Wearing', 'Size', 'Small', 'bottom']
['self-portrait', 'open-back', 'ruched', 'leopard-print', 'jersey', 'maxi', 'dress', 'cream', 'black', 'jersey', 'button', 'fastening', 'keyhole', 'back', '96', 'viscose', '4', 'elastane', '

['remain', 'birger', 'christensen', 'halyn', 'pussy-bow', 'silk-satin', 'blouse', 'beige', 'silk-satin', 'button', 'fastenings', 'front', '100', 'silk', 'dry', 'clean', 'imported', 'Fits', 'True', 'To', 'Size', 'Take', 'Your', 'Normal', 'Size', 'Cut', 'For', 'Loose', 'Fit', 'Dropped', 'Shoulder', 'Seam', 'Lightweight', 'Fluid', 'Non-Stretchy', 'Fabric', 'Model', 'Is', '180Cm', '5', '11', 'And', 'Is', 'Wearing', 'Dk', '34', 'top']
['remain', 'birger', 'christensen', 'halyn', 'pussy-bow', 'silk-satin', 'blouse', 'beige', 'silk-satin', 'button', 'fastenings', 'front', '100', 'silk', 'dry', 'clean', 'imported', 'Fits', 'True', 'To', 'Size', 'Take', 'Your', 'Normal', 'Size', 'Cut', 'For', 'Loose', 'Fit', 'Dropped', 'Shoulder', 'Seam', 'Lightweight', 'Fluid', 'Non-Stretchy', 'Fabric', 'Model', 'Is', '180Cm', '5', '11', 'And', 'Is', 'Wearing', 'Dk', '34', 'top']
['nili', 'lotan', 'makenna', 'cashmere', 'cardigan', 'camel', 'cashmere', 'slips', '100', 'cashmere', 'dry', 'clean', 'imported', 'C

['crew', 'pointed-toe', 'sadie', 'boots', 'suede', 'meet', 'pointy-toe', 'version', 'favorite', 'everyday', 'ankle', 'boot', 'hits', 'exact', 'right', 'height', 'also', 'great', 'soft', 'suede', 'walkable', 'block', 'heel', 'shoe']
['crew', 'pointed-toe', 'sadie', 'boots', 'suede', 'meet', 'pointy-toe', 'version', 'favorite', 'everyday', 'ankle', 'boot', 'hits', 'exact', 'right', 'height', 'also', 'great', 'soft', 'suede', 'walkable', 'block', 'heel', 'shoe']
['crew', 'pointed-toe', 'sadie', 'boots', 'suede', 'meet', 'pointy-toe', 'version', 'favorite', 'everyday', 'ankle', 'boot', 'hits', 'exact', 'right', 'height', 'also', 'great', 'soft', 'suede', 'walkable', 'block', 'heel', 'shoe']
['veronica', 'beard', 'schoolboy', 'dickey', 'jacket', 'schoolboy', 'dickey', 'jacket', 'nod', 'prep', 'school', 'style', 'available', 'lightweight', 'linen', 'ideal', 'petite', 'body', 'type', 'tailored', 'jacket', 'designed', 'slightly', 'shrunken', 'silhouette', 'three-quarter', 'length', 'sleeves', 

['fleur', 'du', 'mal', 'pvc', 'mini', 'skirt', 'black', 'pvc', 'zip', 'fastening', 'along', 'back', '100', 'polyurethane', 'backing', '100', 'polyester', 'lining', '97', 'polyester', '3', 'spandex', 'imported', 'Fits', 'True', 'To', 'Size', 'Take', 'Your', 'Normal', 'Size', 'Mini-Length', 'Designed', 'To', 'Be', 'Worn', 'At', 'The', 'Waist', 'Lightweight', 'Non-Stretchy', 'Fabric', 'Model', 'Is', '180Cm', '5', '11', 'And', 'Is', 'Wearing', 'Us', '2', 'bottom']
['fleur', 'du', 'mal', 'pvc', 'mini', 'skirt', 'black', 'pvc', 'zip', 'fastening', 'along', 'back', '100', 'polyurethane', 'backing', '100', 'polyester', 'lining', '97', 'polyester', '3', 'spandex', 'imported', 'Fits', 'True', 'To', 'Size', 'Take', 'Your', 'Normal', 'Size', 'Mini-Length', 'Designed', 'To', 'Be', 'Worn', 'At', 'The', 'Waist', 'Lightweight', 'Non-Stretchy', 'Fabric', 'Model', 'Is', '180Cm', '5', '11', 'And', 'Is', 'Wearing', 'Us', '2', 'bottom']
['fleur', 'du', 'mal', 'pvc', 'mini', 'skirt', 'black', 'pvc', 'zip', 

['st', 'agni', 'copain', 'linen', 't-shirt', 'classic', 'layering', 'piece', 'st', "agni's", 'copain', 'tee', 'made', 'airy', 'linen', 'cut', 'relaxed', 'fit', 'detailed', 'elongated', 'short', 'sleeves', 'favorite', 'street', 'style', 'looks', 'Slip', 'On', 'Composition', '100', 'Linen', 'Machine', 'Wash', 'Imported', 'Model', 'Is', 'Wearing', 'Sample', 'Size', 'Os', 'This', 'Style', 'Fits', 'Most', 'Sizes', 'top']
['st', 'agni', 'copain', 'linen', 't-shirt', 'classic', 'layering', 'piece', 'st', "agni's", 'copain', 'tee', 'made', 'airy', 'linen', 'cut', 'relaxed', 'fit', 'detailed', 'elongated', 'short', 'sleeves', 'favorite', 'street', 'style', 'looks', 'Slip', 'On', 'Composition', '100', 'Linen', 'Machine', 'Wash', 'Imported', 'Model', 'Is', 'Wearing', 'Sample', 'Size', 'Os', 'This', 'Style', 'Fits', 'Most', 'Sizes', 'top']
['st', 'agni', 'copain', 'linen', 't-shirt', 'classic', 'layering', 'piece', 'st', "agni's", 'copain', 'tee', 'made', 'airy', 'linen', 'cut', 'relaxed', 'fit', 

['far', 'diana', 'crystal-embellished', 'croc-effect', 'leather', 'pumps', "far's", 'slingback', 'diana', 'pumps', 'sophisticated', 'former', 'princess', 'wales', 'set', 'atop', 'sculptural', 'crystal-embellished', 'heel', "they're", 'crafted', 'croc-embossed', 'leather', 'tapered', 'square-toe', 'let', 'peek', 'tailoring', 'straight-leg', 'jeans', 'Heel', 'Height', 'Measures', 'Approximately', '50Mm', '2', 'Inches', 'Slip', 'On', 'Composition', 'Leather', 'Imported', 'Heel', 'Measures', 'Approx', '2', 'Highthis', 'Style', 'Fits', 'True', 'To', 'Sizewe', 'Suggest', 'Taking', 'Your', 'Normal', 'Size', 'shoe']
['far', 'diana', 'crystal-embellished', 'croc-effect', 'leather', 'pumps', "far's", 'slingback', 'diana', 'pumps', 'sophisticated', 'former', 'princess', 'wales', 'set', 'atop', 'sculptural', 'crystal-embellished', 'heel', "they're", 'crafted', 'croc-embossed', 'leather', 'tapered', 'square-toe', 'let', 'peek', 'tailoring', 'straight-leg', 'jeans', 'Heel', 'Height', 'Measures', 'Ap

['sam', 'edelman', '56mm', 'navigator', 'aviator', 'sunglasses', 'slim', 'metal', 'frames', 'tortoiseshell', 'accents', 'bring', 'touch', 'sophistication', 'classic', 'aviators', '56Mm', 'Lens', 'Width', '19Mm', 'Bridge', 'Width', '135Mm', 'Temple', 'Length', '100', 'Uv', 'Protection', 'Adjustable', 'Nonslip', 'Nose', 'Pads', 'Metal', 'Imported', 'accessory', '2']
['sam', 'edelman', 'dori', 'pump', 'kitten', 'heel', 'impeccably', 'pointed', 'toe', 'make', 'pump', 'instant', 'classic', 'True', 'To', 'Size', '2', 'Heel', 'Textile', 'Or', 'Leather', 'Upper', 'Synthetic', 'Lining', 'And', 'Sole', 'Imported', 'Women', 'Shoes', 'shoe']
['sam', 'edelman', 'dori', 'pump', 'kitten', 'heel', 'impeccably', 'pointed', 'toe', 'make', 'pump', 'instant', 'classic', 'True', 'To', 'Size', '2', 'Heel', 'Textile', 'Or', 'Leather', 'Upper', 'Synthetic', 'Lining', 'And', 'Sole', 'Imported', 'Women', 'Shoes', 'shoe']
['sam', 'edelman', 'dori', 'pump', 'kitten', 'heel', 'impeccably', 'pointed', 'toe', 'make'

In [14]:
df.head()

,index,product_id,brand,product_full_name,description,brand_category,details,outfit_id,outfit_item_type,new_column,output_name
0,0,01DVA59VHYAPT4PVX32NXW91G5,tibi,juan embossed mules,tibi's juan embossed mules are made from shiny...,women shoes mules,As Seen On The Pre-Fall ‘19 Runway Heel Measur...,01DVA879D7TQ59VPTTGCMJWWSK,shoe,"[tibi, juan, embossed, mule, tibi's, juan, emb...",shoe:juan embossed mules(01DVA59VHYAPT4PVX32NX...
1,3,01DVA4XY7A0QMMSK3V3SBR52J9,alexandre birman,clarita bow-embellished suede sandals,alexandre birman's 'clarita' sandals have quic...,women shoes sandals,Heel Height Measures Approximately 50Mm / 2 In...,01DVA8GAYP45BCEMYGEK7FXGDQ,shoe,"[alexandre, birman, clarita, bow-embellished, ...",shoe:clarita bow-embellished suede sandals(01D...
2,4,01DVA4XY7A0QMMSK3V3SBR52J9,alexandre birman,clarita bow-embellished suede sandals,alexandre birman's 'clarita' sandals have quic...,women shoes sandals,Heel Height Measures Approximately 50Mm / 2 In...,01DWJE4FDNYRV6ZJBG25HJFYY2,shoe,"[alexandre, birman, clarita, bow-embellished, ...",shoe:clarita bow-embellished suede sandals(01D...
3,9,01DVBP9AHVQTZXJSBNJ0N2NYJP,khaite,leather ankle boots,heel measures approximately 50mm/ 2 inches bla...,shoes boots ankle,"Fits True To Size, Take Your Normal Size Ital...",01DVBPKBK15VKQYKXV1YWR5PZ1,shoe,"[khaite, leather, ankle, boot, heel, measure, ...",shoe:leather ankle boots(01DVBP9AHVQTZXJSBNJ0N...
4,10,01DVBP9AHVQTZXJSBNJ0N2NYJP,khaite,leather ankle boots,heel measures approximately 50mm/ 2 inches bla...,shoes boots ankle,"Fits True To Size, Take Your Normal Size Ital...",01DVBPKBK1CN1AMKDTSKJKGY7S,shoe,"[khaite, leather, ankle, boot, heel, measure, ...",shoe:leather ankle boots(01DVBP9AHVQTZXJSBNJ0N...


## Define functions for different kinds of input

### Recommend by product_id using fuzzy wuzzy

In [15]:
import random
import json
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import warnings;
warnings.filterwarnings('ignore')
from fuzzywuzzy import process
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
def recommend_id(test):
    strOptions =list(set(df['product_id'].to_list()))
    str2Match=product_id
    Ratios = process.extract(str2Match,strOptions)
    highest = process.extractOne(str2Match,strOptions)
    final_prod=highest[0]
    outfit_code=df.loc[df['product_id']==final_prod]['outfit_id'].to_list()
    outfit_code=outfit_code[0]
    final_result=df.loc[df['outfit_id']==outfit_code]['output_name'].to_frame().reset_index(drop=True)
    return final_result

### Recommend by description using TFIDF

In [17]:
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models, similarities

In [18]:
def clean(words):
    word = words.replace("[", "").replace("\'", "").replace("\"", "").replace(" ", "").split(",")
    return word

In [19]:
def recommend_description(test):
    test = test.lower()
    data = list(df.new_column)
    d = []
    for words in data:
        words = str(words)
        d.append(words)
    
    # creating test_model and dictionary
    test_model = [[word for word in clean(words)] for words in d]
    dictionary = corpora.Dictionary(test_model,prune_at=2000000)
    
    # constructing corpus
    corpus_model= [dictionary.doc2bow(test) for test in test_model]
    tfidf_model = models.TfidfModel(corpus_model)
    
    # constructing tfidf based on processed corpus
    corpus_tfidf = tfidf_model[corpus_model]
    
    # creating the bag of words and calculating tfidf
    test_bow = dictionary.doc2bow([word for word in word_tokenize(test)])
    test_tfidf = tfidf_model[test_bow]
    
    # calculating similarities between test and original data
    index = similarities.MatrixSimilarity(corpus_tfidf)
    sims = pd.DataFrame(index[test_tfidf])
    sims.columns = ["similaritie"]
    sims["information"] = data
    sims = sims[sims["similaritie"] <= 0.98]
    sims = sims.sort_values(by="similaritie", ascending=False).head(1)
    
    # get the product's id with the highest similarity
    target_product = list(sims["information"])[0]
    
    # get the outfit id of the target product
    outfitid = list(df[[v == target_product for v in df['new_column'].tolist()]].outfit_id)[0]
    
    # return all products with the same outfit id
    target = df[df["outfit_id"] == outfitid ]

    return target

## Try it!

In [20]:
test = input("Enter your content:")

Enter your content:Sexy silky, a-line mini skirt zipperBenson skirt


In [21]:
if test.startswith( '01' ):
    result = recommend_id(test)
else:
    result = recommend_description(test)

result

,index,product_id,brand,product_full_name,description,brand_category,details,outfit_id,outfit_item_type,new_column,output_name
2184,6656,01DPET2NWSA221STZF740BZ9SW,veronica beard,ashlynn blouse,our silk-chiffon ashlynn blouse is updated thi...,tops,,01DQ63P636Q4BQVCKT6Z4S41G5,top,"[veronica, beard, ashlynn, blouse, silk-chiffo...",top:ashlynn blouse(01DPET2NWSA221STZF740BZ9SW)
3974,12110,01DPKMGJ33SDFXM7XHGPQJWQ12,reformation,benson skirt,sexy silky. this is an a-line mini skirt with ...,skirts,,01DQ63P636Q4BQVCKT6Z4S41G5,bottom,"[reformation, benson, skirt, sexy, silky, a-li...",bottom:benson skirt(01DPKMGJ33SDFXM7XHGPQJWQ12)
4794,14318,01DPCRZWX4S2Z8Q5HYDFM4HNEG,j.crew,pointed-toe flats in suede,since suede is the ultimate transitional finis...,shoes,,01DQ63P636Q4BQVCKT6Z4S41G5,shoe,"[crew, pointed-toe, flat, suede, since, suede,...",shoe:pointed-toe flats in suede(01DPCRZWX4S2Z8...
